In [1]:
# external modules
import sys
import random
import numpy as np
import pandas as pd
import collections
import matplotlib.pyplot as plt
import japanize_matplotlib
from matplotlib.dates import DateFormatter
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm

In [5]:
# read data
df = pd.read_csv("./../data/processed/tidy_df.csv", index_col=[0, 1, 2])
df.describe()

,EPS,INV,AR,CAPX,GM,SA,ETR,LF
count,57564.000000,57564.000000,57564.000000,57564.000000,57564.000000,57564.000000,5.756400e+04,57564.000000
mean,66.198968,774.840637,1261.101340,171.051444,502.457637,377.238859,NaN,16.183696
std,684.969761,6907.588421,7908.071094,2293.751793,3469.166345,2615.653336,NaN,0.788720
min,-37300.500000,-1248.280315,-8093.573906,-3424.537248,-30368.482179,-1970.903771,-inf,11.628080
25%,3.140000,89.375920,176.410138,9.285142,51.975567,39.088947,2.332926e-01,15.645074
50%,14.655000,192.460466,351.561040,24.861669,114.172826,82.605701,3.296066e-01,16.132472
75%,36.742500,410.148638,709.758379,65.351625,219.020289,163.972893,4.146341e-01,16.665310
max,31202.830000,404482.462266,293004.896357,143224.389801,104813.929669,87333.560942,inf,22.530918


In [6]:
df.loc[pd.IndexSlice[:, :, :], :]

決算発表日 事業年度開始年月日［３ヵ月］   事業年度終了年月日        EPS         INV  \
企業名   会計年度 四半期                                                                 
あらた   2008 Q1   2008-08-12     2008-04-01  2008-06-30   2.660000  400.874539   
           Q2   2008-11-12     2008-07-01  2008-09-30   2.420000  354.438952   
           Q3   2009-02-12     2008-10-01  2008-12-31   3.140000  463.063689   
           Q4   2009-05-15     2009-01-01  2009-03-31  -5.480000  352.113272   
      2009 Q1   2009-08-10     2009-04-01  2009-06-30   6.780000  342.920625   
...                    ...            ...         ...        ...         ...   
ｅＢＡＳＥ 2019 Q4   2020-05-15     2020-01-01  2020-03-31  18.657982    0.042405   
      2020 Q1   2020-07-31     2020-04-01  2020-06-30   2.010000    0.084809   
           Q2   2020-10-30     2020-07-01  2020-09-30   3.810000    0.086835   
           Q3   2021-01-29     2020-10-01  2020-12-31   3.070000    0.108546   
           Q4   2021-05-14            NaN  2021-03-31   9.989365    0.108580   

                         AR       CAPX          GM          SA       ETR  \
企業名   会計年度 四半期                                                             
あらた   2008 Q1   1097.393673   9.570469  277.799816  282.517228  0.667780   
           Q2    947.010117  13.330285  245.357228  247.716689  0.645914   
           Q3   1227.472337  24.209869  257.865737  255.535304  0.734463   
           Q4    937.498755  27.924985  224.785468  237.362369 -0.300633   
      2009 Q1    992.622168   5.478945  258.643522  255.350823  0.546833   
...                     ...        ...         ...         ...       ...   
ｅＢＡＳＥ 2019 Q4     40.920347   0.657270   38.291268   11.194789  0.282680   
      2020 Q1     12.615340   0.148416    8.268878    5.533788  0.321168   
           Q2     11.527380   0.151962   11.201748    5.861380  0.308300   
           Q3     11.744667   0.314783   10.181606    5.839770  0.315534   
           Q4     18.328314   0.314882   19.848434    6.015335  0.281931   

                       LF  
企業名   会計年度 四半期             
あらた   2008 Q1   17.678254  
           Q2   17.681603  
           Q3   17.765127  
           Q4   17.575437  
      2009 Q1   17.707993  
...                   ...  
ｅＢＡＳＥ 2019 Q4   15.012871  
      2020 Q1   14.510928  
           Q2   14.653348  
           Q3   14.605000  
           Q4   14.970051  

[57564 rows x 11 columns]

In [16]:
df[
    df["ETR"] == np.inf
]

決算発表日 事業年度開始年月日［３ヵ月］   事業年度終了年月日    EPS  \
企業名        会計年度 四半期                                                 
タカキタ       2010 Q3   2011-02-14     2010-10-01  2010-12-31  -1.80   
           2018 Q3   2019-01-31     2018-10-01  2018-12-31  -1.12   
チタン工業      2016 Q3   2017-02-10     2016-10-01  2016-12-31  -0.03   
ハウス　オブ　ローゼ 2016 Q4   2017-05-12     2017-01-01  2017-03-31  -2.86   
ホクシン       2011 Q2   2011-10-28     2011-07-01  2011-09-30  -0.10   
住友精密工業     2020 Q3   2021-02-12     2020-10-01  2020-12-31 -50.78   
佐藤商事       2008 Q3   2009-02-05     2008-10-01  2008-12-31  -3.72   
広島ガス       2012 Q3   2013-02-08     2012-10-01  2012-12-31  -0.54   
新日本理化      2020 Q2   2020-11-12     2020-07-01  2020-09-30  -0.51   
日本フエルト     2020 Q3   2021-01-29     2020-10-01  2020-12-31  -1.38   
東京製綱       2008 Q3   2009-02-09     2008-10-01  2008-12-31  -1.03   
盟和産業       2018 Q2   2018-11-13     2018-07-01  2018-09-30  -3.18   
石塚硝子       2020 Q2   2020-10-22     2020-06-21  2020-09-20 -28.76   
神戸電鉄       2011 Q2   2011-10-27     2011-07-01  2011-09-30  -0.10   
秀英予備校      2014 Q3   2015-02-06     2014-10-01  2014-12-31  -5.10   
豊和工業       2016 Q4   2017-05-12     2017-01-01  2017-03-31 -11.80   

                             INV           AR        CAPX          GM  \
企業名        会計年度 四半期                                                     
タカキタ       2010 Q3     96.292613   100.362535   11.690200   32.212996   
           2018 Q3    140.082384   168.879506   17.607879   37.991383   
チタン工業      2016 Q3     81.522512    54.503433    6.796312    7.643774   
ハウス　オブ　ローゼ 2016 Q4    322.166387   209.036012    4.890975  498.028831   
ホクシン       2011 Q2     47.327128    48.102983    1.463544   13.753785   
住友精密工業     2020 Q3   5735.885455  3096.418133  165.734434  453.927150   
佐藤商事       2008 Q3   1036.078375  2541.916832  108.299913  142.465957   
広島ガス       2012 Q3     74.905879   122.752052   60.384176   94.352924   
新日本理化      2020 Q2    121.307712   187.231734   12.940741   26.149684   
日本フエルト     2020 Q3    157.551700   214.003791   11.096299   35.833437   
東京製綱       2008 Q3    111.979308   123.296765   13.373314   24.786393   
盟和産業       2018 Q2    549.316668  1464.558791  150.112277  208.528948   
石塚硝子       2020 Q2   4353.048424  3471.067589  436.929292  655.274494   
神戸電鉄       2011 Q2     70.165829     9.365381    2.962392   13.166185   
秀英予備校      2014 Q3     13.860489    49.480455   12.370114   99.706098   
豊和工業       2016 Q4    327.235614   489.335482   15.059549   57.681668   

                             SA  ETR         LF  
企業名        会計年度 四半期                              
タカキタ       2010 Q3    28.056481  inf  15.363327  
           2018 Q3    35.389234  inf  15.551941  
チタン工業      2016 Q3     7.776709  inf  15.758489  
ハウス　オブ　ローゼ 2016 Q4   489.735439  inf  14.950185  
ホクシン       2011 Q2    11.426222  inf  16.563339  
住友精密工業     2020 Q3   432.194585  inf  15.663898  
佐藤商事       2008 Q3   103.188820  inf  18.060010  
広島ガス       2012 Q3    94.675063  inf  16.402321  
新日本理化      2020 Q2    27.732075  inf  16.296745  
日本フエルト     2020 Q3    35.571113  inf  15.194993  
東京製綱       2008 Q3    17.867513  inf  16.111040  
盟和産業       2018 Q2   206.815012  inf  16.133304  
石塚硝子       2020 Q2   644.046677  inf  15.878857  
神戸電鉄       2011 Q2     9.141804  inf  15.644541  
秀英予備校      2014 Q3    44.711255  inf  15.156932  
豊和工業       2016 Q4    61.516461  inf  15.541256

In [17]:
df[
    df["ETR"] == -np.inf
]

,,,決算発表日,事業年度開始年月日［３ヵ月］,事業年度終了年月日,EPS,INV,AR,CAPX,GM,SA,ETR,LF
企業名,会計年度,四半期,,,,,,,,,,,
シナネンホールディングス,2008,Q3,2009-01-30,2008-10-01,2008-12-31,0.09,47.982549,405.320229,14.255574,116.446013,98.703024,-inf,17.621245
ゼリア新薬工業,2017,Q4,2018-05-11,2018-01-01,2018-03-31,4.01,185.208120,308.443133,20.170399,211.364445,206.583606,-inf,15.957153
日本基礎技術,2008,Q2,2008-11-13,2008-07-01,2008-09-30,0.57,217.217006,121.498419,2.314924,16.800734,14.626109,-inf,16.188784


In [19]:
57564 / (13 * 4)

1107.0